## 1. Data Importing

In [ ]:
import mysql.connector

myDB = {
    "user": input("user: "),
    "password": input("password: "),
    "host": input("host: "),
    "buffered": True
}
connect = mysql.connector.connect(**myDB)
cursor = connect.cursor()

In [ ]:
# create database
cursor.execute("CREATE DATABASE vaccinations")
cursor.execute("USE vaccinations")

In [ ]:
import pandas as pd
df = pd.read_csv("covid19_vacc_data.csv")

In [ ]:
# create table
table = "vaccination_data"
columns = ", ".join([f"`{col}` TEXT" for col in df.columns])
create_table_query = f"CREATE TABLE {table} ({columns})"
cursor.execute(create_table_query)

In [ ]:
# insert data from df into table
for index, row in df.iterrows():
    # replace NaN with NULL
    row = row.where(pd.notnull(row), None)
    placeholders = ", ".join(["%s"] * len(row))
    insert_query = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))

connect.commit()

## 2. Data Cleaning

In [ ]:
# update 1: changing rows value where column "Continent" is NULL
update1 = """
UPDATE vaccination_data
SET continent = CASE
	WHEN location = 'Africa' THEN 'Africa'
	WHEN location = 'Asia' THEN 'Asia'
	WHEN location = 'Europe' THEN 'Europe'
	WHEN location = 'North America' THEN 'North America'
	WHEN location = 'Oceania' THEN 'Oceania'
	WHEN location = 'South America' THEN 'South America'
	WHEN location = 'World' THEN 'World'
	ELSE continent
END
"""

cursor.execute(update1)
connect.commit()

In [ ]:
# update 2: Countries with incorrect international names
update2 = """
UPDATE vaccination_data
SET location = CASE
	WHEN location = 'Sint Maarten (Dutch part)' THEN 'Sint Maarten'
	WHEN location = 'Micronesia (country)' THEN 'Micronesia'
	ELSE location
END
"""

cursor.execute(update2)
connect.commit()

In [ ]:
# selecting necessary data for visualization
"""
note that the lists excluding sovereign nations or individual countries that
are not part of islands, regions, or territories
"""

data = """
SELECT distinct location AS geo_entity, continent,
	CAST(date AS date) AS date,
	CAST(population AS signed) AS population,
	CAST(gdp_per_capita AS float) AS gdp_per_capita,
	CAST(people_vaccinated AS signed) AS people_vaccinated,
	CAST(people_fully_vaccinated AS signed) AS people_fully_vaccinated,
	CAST(total_vaccinations_per_hundred AS float) AS total_vaccination_per_hundred
FROM vaccination_data
WHERE location is not null
	AND location not in('England', 'European Union', 'High income', 'Lower middle income',
	'Low income', 'Northern Ireland', 'Scotland', 'Upper middle income', 'Wales',
	'French Guiana', 'Mayotte', 'Guam', 'Saint Pierre and Miquelon', 'Puerto Rico',
	'Martinique', 'Saint Barthelemy', 'Northern Mariana Islands', 'Western Sahara',
	'Guadeloupe', 'United States Virgin Islands', 'Saint Martin (French part)', 'Reunion',
	'American Samoa', 'Northern Cyprus')
	AND date is not null
ORDER BY location, date
"""

cursor.execute(data)
result = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=columns)

## 2.5. Saving Data

In [ ]:
df.to_csv("covid19_vacc_data_clean.csv", index=False)

## 3. Visualization Dashboard

Here I use *Tableau* for the dashboard


In [ ]:
import webbrowser

url = "https://public.tableau.com/app/profile/erich.dewantara/viz/GlobalCovid-19Vaccinations_16993772434440/VaccinationDashboard"
webbrowser.open(url)